In [7]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join(os.pardir, os.pardir, 'src'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from modules import dataloading as dl
from modules import cleaning as c
import pandas as pd
targetdir = '../../src/data/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# import data'
apr_2020_df = dl.CPS_raw(targetdir, ['apr2020'], None)
jul_2020_df = dl.CPS_raw(targetdir, ['jul2020'], None)
# jan_feb_2020_df = dl.CPS_raw(targetdir, ['jan2020', 'feb2020'], None)


In [41]:
apr_2020_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129382 entries, 0 to 129381
Columns: 391 entries, HRHHID to IND_ID_FINAL
dtypes: int64(387), object(4)
memory usage: 386.0+ MB


# We decided on which columns to keep and which to discard

The target variable will be change in employment status from having a job to not. We could also make a conditional where if the they WHERE full time (40 hours+) but now are not, that could also represent a change in the employment status.


But if our question is: How many people lost full time work during the first wave of the pandemic, The data set becomes much simpler.

In [68]:
to_keep = [
    'PREMPNOT', # EMPLOY TARGET VARIABLE
    'PEMLR', # TARGET VARIABLE 1-6 employment status
    'PRPERTYP', # Type of person recode 
    'HEHOUSUT', # type of housing unit to dummy
    'HETENURE', # rent or morgage status
    'HWHHWGT', # Household weight
    'GESTFIPS', # state codes
    "GTMETSTA", # Metropolitan or not
    'HEFAMINC', # total family income
    "HRNUMHOU", # total number of people living in the house hold
    'HRHTYPE', # household type eg civilian or married etc
    'PERRP', # relationship to reference, -1, 40 and 41
    'PRTAGE', # person's age
    'PEMARITL', # marital status
    'PESPOUSE', # line number of spouse (-1) == no spouse
    'PESEX', # gender 1 == male, 2 == female
    'PEEDUCA', # level of education see dict for coding
    'PTDTRACE', # race composition of the house. See data dict
    "PEHSPNON", # hispanic or not hispanic
    'PUCHINHH', # change in household composition. See data dict
    'PENATVTY', # country of birth ie US born or not
    "PEMNTVTY", # mothers country of birth, 
    "PEFNTVTY", # fathers country of birth ie are they a first generation imiigrant?
    "PRCITSHP", # citezen status
    "PRINUSYR", # Year since immigration -1== us born, else coded by decade
    "PUABSOT", # Did you have a job last week TARGET POTENTIAL
    "PULAY", # Where you on layoff from your job last week?
    "PEABSRSN", # reason for not being at work the day before.
    "PRHRUSL", # Hours at work, dummy into full time or not full time
    "HUBUS", # Does anyone have a business or a farm?
    "PEMJOT", # Do you have more than 1 job?
    "PEMJNUM", # how many jobs do you have?
    "PEHRFTPT", # Do you normally spend more than 35 hours a week at your main job?
    "PEHRRSN2", # what is the main reason you do not want to work 35 hours. Speaks to motivation of keeping job.
    "PEHRACTT", # sum of hours worked between all jobs
    "PRCIVLF", # in the civiliarn labour force = 1 
    "PRAGNA", # Agricultural industry yes or no
    "PRMJIND1", # industry recode
    "PEERNUOT", # Do you usually recieve tips at your work
    "PEERNHRY", # hourly vs non-hourly status
    "PRERNWA", # weekly earnings
    "PEERNLAB", # part of a labour union
    "PRNLFSCH", # In school or not
    "PRNMCHLD", # number of children less than 18 years old (-1 not a parent, 0-99 number of children)
    "PECYC", # How much college credit in years has the reference person recieved?
    "PEIO1ICD", # Inustry code for primary jobs lin1
    "PEIO1OCD", # Occupation code for primary jobs lin1
    "PECERT1", # Do you have a professional certification issued at state or federal level.
    "PUIODP2", # did your job change last month
]

In [69]:
len(to_keep)

49

In [1]:
#"PRMJIND1" & PRMJIND2,  industry codes Primary business industry job 1

"""
1 Agriculture, forestry, fishing, and hunting\
2 Mining
3 Construction
4 Manufacturing
5 Wholesale and retail trade
6 Transportation and utilities      
7 Information
8 Financial activities
9 Professional and business services
10 Educational and health services
11 Leisure and hospitality
12 Other services
13 Public administration
14 Armed Forces"""

'\n1 Agriculture, forestry, fishing, and hunting2 Mining\n3 Construction\n4 Manufacturing\n5 Wholesale and retail trade\n6 Transportation and utilities      \n7 Information\n8 Financial activities\n9 Professional and business services\n10 Educational and health services\n11 Leisure and hospitality\n12 Other services\n13 Public administration\n14 Armed Forces'

In [67]:
"""U.S. Census Bureau
2020 Current Population Survey Public Use Industry Code List					
					
	*  0010 - 0560  Agriculture, Forestry, Fishing and Hunting, and Mining				
	*  0770 - 1060  Construction				
	*  1070 - 4060  Manufacturing				
	*  4070 - 4660  Wholesale Trade				
	*  4670 - 6060  Retail Trade				
	*  6070 - 6460, 0570 - 0760  Transportation and Warehousing, and Utilities				
	*  6470 - 6860  Information				
	*  6870 - 7260  Finance and Insurance,  and Real Estate and Rental and Leasing				
	*  7270 - 7790  Professional, Scientific, and Management, and  Administrative and Waste Management Services				
	*  7860 - 8490  Educational Services, and Health Care and Social Assistance				
	*  8560 - 8690  Arts, Entertainment, and Recreation, and  Accommodation and Food Services				
	*  8770 - 9290  Other Services (except Public Administration)				
	*  9370 - 9590  Public Administration				
	*  9890         Military	"""

'U.S. Census Bureau\t\t\t\t\t\n2020 Current Population Survey Public Use Industry Code List\t\t\t\t\t\n\t\t\t\t\t\n\t*  0010 - 0560  Agriculture, Forestry, Fishing and Hunting, and Mining\t\t\t\t\n\t*  0770 - 1060  Construction\t\t\t\t\n\t*  1070 - 4060  Manufacturing\t\t\t\t\n\t*  4070 - 4660  Wholesale Trade\t\t\t\t\n\t*  4670 - 6060  Retail Trade\t\t\t\t\n\t*  6070 - 6460, 0570 - 0760  Transportation and Warehousing, and Utilities\t\t\t\t\n\t*  6470 - 6860  Information\t\t\t\t\n\t*  6870 - 7260  Finance and Insurance,  and Real Estate and Rental and Leasing\t\t\t\t\n\t*  7270 - 7790  Professional, Scientific, and Management, and  Administrative and Waste Management Services\t\t\t\t\n\t*  7860 - 8490  Educational Services, and Health Care and Social Assistance\t\t\t\t\n\t*  8560 - 8690  Arts, Entertainment, and Recreation, and  Accommodation and Food Services\t\t\t\t\n\t*  8770 - 9290  Other Services (except Public Administration)\t\t\t\t\n\t*  9370 - 9590  Public Administration\t\t\

In [20]:
apr_cleaned = c.clean_CPS_df(apr_2020_df)
jul_cleaned = c.clean_CPS_df(jul_2020_df)